In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Colab\ Notebooks/NLP

/content/drive/MyDrive/Colab Notebooks/NLP


In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
SEQ_LENGTH = 40


In [ ]:
 !pip install nltk==3.4

In [ ]:
# -*- coding: utf-8 -*-
import os
from os.path import join, dirname
import itertools
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Activation, TimeDistributed, Dense, LSTM, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import re
from nltk import ngrams
data_path = join('train.txt')
with open(data_path, 'r') as f_r:
    lines = f_r.read().split('\n')

print(len(lines))

NGRAM = 20
BATCH_SIZE = 64
HIDDEN_SIZE = 256
eng_alphabet = "abcdefghijklmnopqrstuvwxyz"
viet_alphabet = "áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ"
digits = "0123456789"
punctuations = " _!\"\',\-\.:;?_\(\)\x00"

pattern = "^[" + "".join((eng_alphabet, viet_alphabet, digits, punctuations)) + "]+$"


2126281


In [ ]:
# def extract_phrases(text):
#     return ''.join(re.findall(r'\w[\w ]+', text))

def extract_phrases(text):
    phs = re.findall(r'\w[\w ]+', text)
    phsr = []
    for p in phs :
        token =  text.split(p)[1]
        if (len(token) > 0):
            token = token[0]
        else :
            phsr.append(p)
            continue                
        if token in pattern:
            p += token
        phsr.append(p)
    return [' '.join(phsr)]

def gen_ngrams(words, n=NGRAM):
    return ngrams(words.split(), n)
    
    
phrases = itertools.chain.from_iterable(extract_phrases(text) for text in lines[:int(len(lines)//2)])
phrases = [p.strip() for p in phrases if len(p.split()) > 1]

list_ngrams = []
for p in tqdm(phrases):
    if not re.match(pattern, p.lower()):
        continue
    list_ngrams.append(p)
del phrases
list_ngrams = list(set(list_ngrams))
print(list_ngrams[0:5])

100%|██████████| 1048493/1048493 [00:03<00:00, 330645.51it/s]


['Theo ông Niệm, với đất ở nên chia gắn liền với đất sản xuất ( từ 2. 000 đến 3. 000 m2)', 'Taeniorhachis repens', 'Người dân phải đi bộ trên mái nhà nhiều tháng nay vì thang máy hư hỏng.', 'Hiện nay, các bác sĩ chẩn đoán bệnh sán lá phổi bằng kỹ thuật miễn dịch ELISA.', 'Thực tế, Nga trong trận lượt đi tại Moscow đã chơi tốt cho đến khi họ có bàn thắng nhân đôi cách biệt ở phút 51.']


In [ ]:
print(len(list_ngrams))

904459


In [ ]:
lines = ['Bộ phim lần đầu được công chiếu tại liên hoan phim Rome 2007 và sau đó được chiếu ở Fairbanks, Alaska ngày 21 tháng 9 năm 2007.']
phrases = itertools.chain.from_iterable(extract_phrases(text) for text in lines)
phrases = [p for p in phrases if len(p.split()) > 1]
phrases


['Bộ phim lần đầu được công chiếu tại liên hoan phim Rome 2007 và sau đó được chiếu ở Fairbanks, Alaska ngày 21 tháng 9 năm 2007.']

In [ ]:
file = open('train.txt', encoding = 'utf8')
raw_text = file.read()
# raw_text = raw_text.lower()

In [ ]:
import string 

eng_alphabet = "abcdefghijklmnopqrstuvwxyz"
viet_alphabet = "áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ"
digits = "0123456789"
punctuations = " _!\"\',\-\.:;?_\(\)\x00"

pattern = "^[" + "".join((eng_alphabet, viet_alphabet, digits, punctuations)) + "]+$"

accented_chars_vietnamese = [
    'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ',
    'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ',
    'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ',
    'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự',
    'í', 'ì', 'ỉ', 'ĩ', 'ị',
    'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ',
    'đ',
]
accented_chars_vietnamese.extend([c.upper() for c in accented_chars_vietnamese])
chars = list(('\x00 _' + string.ascii_letters + string.digits
                 + ''.join(accented_chars_vietnamese) + string.punctuation))


print(chars)
VOCABULARY = len(chars)

int_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_int = dict((c, i) for i, c in enumerate(chars))

['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ', '

In [ ]:
text_length = len(raw_text)
char_length = len(chars)
print("Text length = " + str(text_length))
print("No. of characters = " + str(char_length))

Text length = 199087994
No. of characters = 231


In [ ]:
def make_noise(text):
    x = []
    y = []
    text = ' '*(SEQ_LENGTH - 1) + text
    if (len(text) <= SEQ_LENGTH):
        text = ' '*(SEQ_LENGTH - len(text)) + text
        x_t = text[: SEQ_LENGTH -1]
        x_tt = [char_to_int[char] for char in x_t]
        x.append(x_tt)
        y.append(char_to_int[text[SEQ_LENGTH -1]])
    else :
        if (len(text)>=1500):
            text = text[:1500]
        for i in range(len(text) - SEQ_LENGTH):
            x_t = text[i :i + SEQ_LENGTH - 1]
            x_tt = [char_to_int[char] for char in x_t]
            x.append(x_tt)
            y.append(char_to_int[text[i + SEQ_LENGTH - 1]])
    return x,y


In [ ]:
import numpy as np
def generate_data(data, batch_size=64):
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    cur_index = 0
    while True:
        x = []
        y = []
        for i in range(batch_size):
            w, l = make_noise(data[cur_index])     
            x += w
            y += l
            cur_index += 1

            if cur_index > len(data) - 1:
                cur_index = 0
        # length = len(x)
        x = np.array(x)
        x = np. reshape(x, (x.shape[0], x.shape[1], 1))
        x = x/float(VOCABULARY)

        y = np.array(y)
        y = np_utils.to_categorical(y, num_classes=VOCABULARY)
        yield x, y



In [ ]:
train_data, valid_data = train_test_split(list_ngrams, test_size=0.2)
print(len(train_data))
train_generator = generate_data(train_data, batch_size=BATCH_SIZE)
validation_generator = generate_data(valid_data, batch_size=BATCH_SIZE)

723567


In [ ]:
def buildmodel(VOCABULARY):
    model = Sequential()
    model.add(LSTM(256, input_shape = (SEQ_LENGTH, 1), return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(VOCABULARY, activation = 'softmax'))
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
model = buildmodel(VOCABULARY)
checkpoint = ModelCheckpoint(filepath=os.path.join('trained_model/model1_{val_loss:.4f}_{val_accuracy:.4f}.hdf5'),
                               save_best_only=True, verbose=1)
early = EarlyStopping(patience=2, verbose=1)
BATCH_SIZE = 64
history = model.fit_generator(train_generator, steps_per_epoch=len(train_data)//BATCH_SIZE, epochs=5,
                    validation_data=validation_generator, validation_steps=len(valid_data)//BATCH_SIZE,
                    callbacks=[checkpoint, early])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
11305/11305 [==============================] - 10055s 889ms/step - loss: 2.6046 - accuracy: 0.3422 - val_loss: 1.8786 - val_accuracy: 0.4877

Epoch 00001: val_loss improved from inf to 1.87857, saving model to trained_model/model1_1.8786_0.4877.hdf5
Epoch 2/5
11305/11305 [==============================] - 10031s 887ms/step - loss: 1.9368 - accuracy: 0.4739 - val_loss: 1.7490 - val_accuracy: 0.5204

Epoch 00002: val_loss improved from 1.87857 to 1.74896, saving model to trained_model/model1_1.7490_0.5204.hdf5
Epoch 3/5
11305/11305 [==============================] - 10031s 887ms/step - loss: 1.8386 - accuracy: 0.5000 - val_loss: 1.6879 - val_accuracy: 0.5369

Epoch 00003: val_loss improved from 1.74896 to 1.68793, saving model to trained_model/model1_1.6879_0.5369.hdf5
Epoch 4/5
 1610/11305 [===>..........................] - ETA: 2:12:10 - loss: 1.7956 - accuracy: 0.5115

In [ ]:
filename = 'trained_model/model1_1.9452_0.4711.hdf5'
model = buildmodel(VOCABULARY)
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [ ]:
initial_text = 'Trong chiến thắng bất ngờ của đội tuyển Việt Nam, '# we sat here we two and we said how we wish we had something to do.'
print(len(initial_text))
initial_text = [char_to_int[c] for c in initial_text[-50:]]

In [ ]:
GENERATED_LENGTH = 1000
test_text = initial_text
generated_text = []

for i in range(100):
    X = np.reshape(test_text, (1, SEQ_LENGTH, 1))
    next_character = model.predict(X/float(VOCABULARY))
    index = np.argmax(next_character)
    generated_text.append(int_to_char[index])
    test_text.append(index)
    test_text = test_text[1:]

In [ ]:
generated_text

In [ ]:
print(''.join(generated_text))